In [3]:
import sqlite3
from sqlite3 import Error

# Creates database connection to an SQLite database.
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

# Creates Students, Enrolled, and Courses tables at the specified database connection, if they don't exist already.
def create_tables(conn):
    if conn is not None:
        sql_create_students_table = """ CREATE TABLE IF NOT EXISTS Students (
                                            sid integer PRIMARY KEY,
                                            sname text NOT Null,
                                            age integer);"""
        sql_create_enrolled_table = """CREATE TABLE IF NOT EXISTS Enrolled (
                                            sid integer,
                                            cid integer,
                                            grade text,
                                            FOREIGN KEY (sid) REFERENCES Students (sid)
                                            FOREIGN KEY (cid) REFERENCES Courses (cid)
                                            );"""
        sql_create_courses_table = """ CREATE TABLE IF NOT EXISTS Courses (
                                            cid integer PRIMARY KEY,
                                            cname text,
                                            credits integer);"""
        try:
            c=conn.cursor()
            c.execute(sql_create_students_table)
            c.execute(sql_create_enrolled_table)
            c.execute(sql_create_courses_table)
        except Error as e:
            print(e)
    else:
        print("Error! Can not create the database connection.")

# Creates a new Student record and adds it to the Student table.
def create_student(conn, values):
    sql = ''' INSERT INTO Students(sid, sname, age) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Creates a new Enrollment record and adds it to the Enrolled table.
def create_enrollment(conn, values):
    sql = ''' INSERT INTO Enrolled(sid, cid, grade) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Creates a new Course record and adds it to the Course table.
def create_course(conn, values):
    sql = ''' INSERT INTO Courses(cid, cname, credits) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Queries the Students table to check if a given sid is present.
def check_if_sid_exists(conn, sid):
    sql = 'SELECT count(*) FROM Students WHERE sid = ?'
    cur = conn.cursor()
    cur.execute(sql, (sid,))
    if(cur.fetchall()[0] < (1,)):
        return False
    else:
        return True

# Queries the Courses table to check if a given cid is present.
def check_if_cid_exists(conn, cid):
    sql = 'SELECT count(*) FROM Courses WHERE cid = ?'
    cur = conn.cursor()
    cur.execute(sql, (cid,))
    if(cur.fetchall()[0] < (1,)):
        return False
    else:
        return True

# Queries the Enrolled table to check if a student has enrolled in a course.
def check_if_sid_is_enrolled_in_cid(conn, sid, cid):
    sql = 'SELECT count(*) FROM Enrolled e WHERE sid = ? and cid = ?'
    values = (sid, cid)
    cur = conn.cursor()
    cur.execute(sql, values)
    if(cur.fetchall()[0] < (1,)):
        return False
    else:
        return True
    
# Queries the Enrolled table to check if a student has enrolled in a course.
def check_if_sid_is_registered_for_all_courses(conn, sid):
    sql1 = 'SELECT count(*) FROM Enrolled e WHERE sid = ?'
    sql2 = 'SELECT count(*) FROM Courses'
    cur1 = conn.cursor()
    cur2 = conn.cursor()
    cur1.execute(sql1, (sid,))
    cur2.execute(sql2)
    if(cur1.fetchall() == cur2.fetchall() and cur2.fetchall() != [0,]):
        return True
    else:
        return False

# Queries the Courses table to check if it is empty.
def check_if_courses_table_is_empty(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM Courses")    
    if(cur.fetchall() == []):
        return True
    else:
        return False

# Populates all tables with sample data.
def populate_tables_with_sample_data(conn):
    # Add sample students.
    student_1 = ('1', 'Hugh G. Rection', '15')        
    create_student(conn, student_1)
    
    # Add sample courses.
    course_1 = ('313', 'Fundamentals of Programming Languages', '3')
    course_2 = ('315', 'Software Engineering I', '3')
    course_3 = ('355', 'Database Systems', '3')
    create_course(conn, course_1)
    create_course(conn, course_2)
    create_course(conn, course_3)
    
    # Add sample enrollments.
    enrollment_1 = ('1', '355', 'A')
    create_enrollment(conn, enrollment_1)
    
    conn.commit()

# Begins the command line interface.
def start_cmd_interface(conn):
    # Print greeting message.
    print("Welcome to MSU Course Registration System!")
    
    # Prompt user to sign in by sid. Reprompt user if sid does not meet constraints.
    while(True):
        try:
            sid = int(input("Enter your student id (or enter -1 to sign up): "))
            if(sid == -1):
                create_new_student(conn)
            elif(check_if_sid_exists(conn, str(sid))):
                break
            else:
                print("Sid does not exist in the database.")
        except:
            print("Sid must be an integer.")

    # Prompt user to select a command and call the appropriate function in response.
    while(True):
        userInput = input("Enter a command: ")
        if(userInput == 'L'):
            select_all_courses(conn)
        elif(userInput == 'E'):
            enroll_in_course(conn, sid)
        elif(userInput == 'W'):
            withdraw_from_course(conn, sid)
        elif(userInput == 'S'):
            search_for_course(conn)
        elif(userInput == 'M'):
            view_my_classes(conn, sid)
        elif(userInput == 'H'):
            show_help_prompt()
        elif(userInput == 'X'):
            print("Goodbye!")
            break
        else:
            print("Not a valid command.")

# Prompts the user to enter their personal information to create a new student record in the Students table.
def create_new_student(conn):  
    # Prompt user to enter an sid. Reprompt user if sid does not meet constraints.
    while(True):
        try:
            sid = int(input("Enter an sid: "))
            if(check_if_sid_exists(conn, str(sid))):
                print("Sid alreay exists.")
            else:
                break
        except:
            print("Sid must be an integer.")
    
    # Prompt user to enter an sname. Reprompt user if sname does not meet constraints.
    while(True):
        sname = str(input("Enter your name: "))
        if(set(sname).issubset(set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ -'))):
            break
        else:
            print("Names must use only letters A to Z.")
            
    # Prompt user to enter an age. Reprompt user if age does not meet constraints.
    while(True):
        try:
            age = int(input("Enter an age: "))
            if(age < 0 or age > 100):
                print("Age must be between 0 and 100.")
            else:
                break
        except:
            print("Age must be an integer.")
    
    # Create a new Student record and add it to the Students table.
    values = (str(sid), str(sname), str(age))
    create_student(conn, values)
    conn.commit()
    
    # Print success message.
    print("Student successfully added.")

# Prints all records in the Courses table.
def select_all_courses(conn):
    # Query Courses table for list of courses.
    cur = conn.cursor()
    cur.execute("SELECT * FROM Courses")
    rows = cur.fetchall()
    
    # Print list of courses.
    if(rows == []):
        print("No courses are available.")
    else:
        for row in rows:
            print(row)

# Enrolls a student in a course.
def enroll_in_course(conn, sid):
    # Check if student is already registered in every available course.
    if(check_if_sid_is_registered_for_all_courses(conn, str(sid))):
        print("You are registered for all available courses.")
    # Check if Courses table is empty.
    elif(check_if_courses_table_is_empty(conn)):
        print("There are no available courses to enroll to.")
    else:
        # Prompt user to enter an cid. Reprompt user if cid does not meet constraints.
        while(True):
            try:
                cid = int(input("Enter a cid: "))
                if(check_if_cid_exists(conn, str(cid))):
                    if(check_if_sid_is_enrolled_in_cid(conn, str(sid), str(cid))):
                        print("Already enrolled in course.")
                    else:
                        break
                else:
                    print("Course does not exist.")
            except:
                print("Cid must be an integer.")
        
        # Create a new Enrollment record and add it to the Enrolled table.
        cur = conn.cursor()
        values = (sid, cid, '')
        create_enrollment(conn, values)
        conn.commit()
        
        # Print success message.
        print("Registered for course successfully.")

# Withdraws a student from a course.
def withdraw_from_course(conn, sid):
    course_id = input("Enter a valid course id: ")
    cur = conn.cursor()
    #Constraint Block
    
    query = "SELECT count(*) FROM Courses WHERE cid = ?"
    sec_query = "SELECT count(*) FROM Enrolled WHERE cid = ?"
    while (True):
        if (cur.execute(query, (course_id,)).fetchall()[0]>(0,)  and cur.execute(sec_query, (course_id,)).fetchall()[0]>(0,)):
            print("Withdrawing from Course")
            break
        else:
            print("Entered Course ID is invalid.")
            course_id = input("Enter a valid course id: ")
    
    #Constraint Block End
                
    values = (sid, course_id)
    sql = 'DELETE FROM Enrolled WHERE sid = ? AND cid = ?'
    #cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()

# Prompts the user to enter a substring to return courses with names similar to the substring.
def search_for_course(conn):
    substring = input("Enter a substring: ")
    sql = 'SELECT * FROM Courses WHERE cname LIKE \"%?%\"'
    cur = conn.cursor()
    cur.execute(sql.replace("?", substring))
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Prints all courses a student is enrolled in.
def view_my_classes(conn, sid):
    sql = 'SELECT c.cid, c.cname, c.credits FROM Enrolled e, Courses c WHERE e.cid = c.cid AND e.sid = ?'
    cur = conn.cursor()
    cur.execute(sql.replace("?", str(sid)))
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Shows a list of functions the user can execute in the command line interface.
def show_help_prompt():
    print("L: List all available courses",
          "\nE: Enroll yourself in a course",
          "\nW: Withdraw yourself from a course",
          "\nS: Search for a course by name",
          "\nM: List your current enrollments",
          "\nH: List executable functions",
          "\nX: Exit application")

# Main method.
def main():
    
    # Create connection to database.
    database = r"DatabaseProject.db"
    conn = create_connection(database)
    
    # Create Students, Enrolled, and Courses tables, if they don't already exist.
    create_tables(conn)
    
    # Populate the tables with sample data...
    # populate_tables_with_sample_data(conn)
    
    # Begin command line interface.
    start_cmd_interface(conn)
        
    # Close connection to database.
    conn.close()

if __name__ == '__main__':
     main()

Welcome to MSU Course Registration System!
Enter your student id (or enter -1 to sign up): 1
Enter a command: L
(313, 'Fundamentals of Programming Languages', 3)
(315, 'Software Engineering I', 3)
(355, 'Database Systems', 3)
Enter a command: M
(355, 'Database Systems', 3)
Enter a command: E
Enter a cid: 313
Registered for course successfully.
Enter a command: E
Enter a cid: 355
Already enrolled in course.
Enter a cid: 315
Registered for course successfully.
Enter a command: E
You are registered for all available courses.
Enter a command: W
Enter a valid course id: 315
Withdrawing from Course
Enter a command: E
Enter a cid: 315
Registered for course successfully.
Enter a command: W
Enter a valid course id: 355
Withdrawing from Course
Enter a command: W
Enter a valid course id: 315
Withdrawing from Course
Enter a command: X
Goodbye!
